# Clustering Neighborhoods in Toronto

Assignment to explore, segment, and cluster the neighborhoods in the city of Toronto. 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Toronto Dataset</a>

2. <a href="#item2">Venues in Downtown Toronto</a>

3. <a href="#item3">Analyze Each Neighborhood in Downtown Toronto</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

## Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Toronto Dataset

Toronto dataset will consist of 5 columns: PostalCode, Borough, Neighborhood, Latitude and Longitude.

* Dataset taken from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
* BeautifulSoup package used to parse the data. 
* Code to retrieve data is here: *******

In [2]:
import pandas as pd

In [3]:
df_toronto_nb = pd.read_csv('toronto_neighborhoods.csv')
df_toronto_nb.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto_nb['Borough'].unique()), df_toronto_nb.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


### Geographical coordinates of  Toronto

In [5]:
# To avoid error GeocoderServiceError: [SSL: CERTIFICATE_VERIFY_FAILED]
import certifi
import ssl
import geopy.geocoders 
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

In [6]:
def get_latitude_longitude(address, agent): 
    geolocator = Nominatim(user_agent=agent)
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude, longitude

In [7]:
latitude, longitude = get_latitude_longitude('Toronto, ON', "on_explorer")
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude)) 

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Toronto map with neighborhoods|

In [8]:
def create_neighborhood_map(latitude, longitude, df_data, zoom=10, color='green'):
    map_view = folium.Map(location=[latitude, longitude], zoom_start=zoom)

    # add markers to map
    for lat, lng, borough, neighborhood in zip(df_data['Latitude'], df_data['Longitude'], df_data['Borough'], df_data['Neighborhood']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_view)  
        
    return map_view

In [9]:
map_toronto = create_neighborhood_map(latitude, longitude, df_toronto_nb)
map_toronto

## Downtown Toronto

In [10]:
down_toronto = df_toronto_nb[df_toronto_nb['Borough']=='Downtown Toronto']
down_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [11]:
print('Downtown Toronto has {} neighborhoods.'.format(down_toronto.shape[0]))

Downtown Toronto has 19 neighborhoods.


In [12]:
latitude_dt, longitude_dt = get_latitude_longitude('Downtown Toronto, ON', "on_explorer")
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude_dt, longitude_dt)) 

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [13]:
map_down_toronto = create_neighborhood_map(latitude_dt, longitude_dt, down_toronto, zoom=12, color='limegreen')
map_down_toronto


## Foursquare API

In [14]:
CLIENT_ID = '0TPBBOGOU5SJALY0M2DW3E4WBO5TXJCMDROYNFDQVYPVQLST'  
CLIENT_SECRET = 'AYQ0CHZTPKTKTDCTFNFPJIMWM4DPHJFDAB3XJJWOP45MF05Q' 
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100 

code copied from NY lab-week3:

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<a id='item2'></a>

## 2. Venues in Downtown Toronto

Getting venues using Foursquare API

In [16]:
down_toronto_venues = getNearbyVenues(names=down_toronto['Neighborhood'],
                                   latitudes=down_toronto['Latitude'],
                                   longitudes=down_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [17]:
print('Downtown Toronto has {} venues.'.format(len(down_toronto_venues['Venue'].unique())))

Downtown Toronto has 716 venues.


In [18]:
down_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues were returned for each neighborhood

In [19]:
down_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,64,64,64,64,64,64
Christie,18,18,18,18,18,18
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


Let's find out how many unique categories can be curated from all the returned venues

In [20]:
print('Downtown Toronto has {} uniques categories in {} neighborhoods'.format(
    len(down_toronto_venues['Venue Category'].unique()),
    len(down_toronto_venues['Neighborhood'].unique())))

Downtown Toronto has 206 uniques categories in 19 neighborhoods


<a id='item3'></a>

## 3. Analyze Each Neighborhood in Downtown Toronto

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(down_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

if 'Neighborhood' in toronto_onehot.columns.tolist():
    toronto_onehot.drop(columns=['Neighborhood'], inplace=True)
    
# add neighborhood column back to dataframe
toronto_onehot.insert(0, 'Neighborhood', down_toronto_venues['Neighborhood'])
toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Cable Car,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [22]:
toronto_onehot.shape

(1231, 206)

### Mean of the frequency of occurrence of each category

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Cable Car,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.017241,0.000000,0.000000,0.0,0.000000,0.017241,0.000000,0.017241,0.034483,0.0,0.000000,0.0,0.017241,0.017241,0.0,0.034483,0.0,0.0,0.017241,0.000000,0.000000,0.000000,0.0,0.017241,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.034483,0.000000,0.000000,0.034483,0.0,0.0,0.0,0.017241,0.051724,0.068966,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.000000,0.017241,0.0,0.0,0.017241,0.0,0.000000,0.0,0.000000,0.0,0.017241,0.000000,0.000000,0.000000,0.0,0.000000,0.017241,0.0,0.0,0.0,0.000000,0.034483,0.000000,0.0,0.017241,0.000000,0.0,0.0,0.017241,0.017241,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.017241,0.017241,0.017241,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.017241,0.0,0.0,0.00000,0.000000,0.0,0.0,0.034483,0.017241,0.017241,0.0,0.017241,0.000000,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.000000,0.000000,0.017241,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.017241,0.0,0.000000,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.000

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Top 7 venues for each neighborhood.

In [25]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Bakery,Italian Restaurant,Farmers Market,Seafood Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Coffee Shop,Plane
2,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Thai Restaurant,Bar,Salad Place
3,Christie,Grocery Store,Café,Park,Coffee Shop,Candy Store,Diner,Restaurant
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio,Pub,Gay Bar


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [26]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 4, 2, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = down_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Restaurant,Café,Theater
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Sushi Restaurant,Diner,Discount Store,Park,Mexican Restaurant,Juice Bar
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Gastropub,Cocktail Bar,American Restaurant,Italian Restaurant,Gym
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Café,Bakery,Italian Restaurant,Farmers Market,Seafood Restaurant


Finally, let's visualize the resulting clusters

In [28]:
# create map
map_clusters = folium.Map(location=[latitude_dt, longitude_dt], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

In [29]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Restaurant,Café,Theater
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Sushi Restaurant,Diner,Discount Store,Park,Mexican Restaurant,Juice Bar
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Gastropub,Cocktail Bar,American Restaurant,Italian Restaurant,Gym
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Café,Bakery,Italian Restaurant,Farmers Market,Seafood Restaurant


In [30]:
# Filter: Neighborhood and Most-common columns
select_cols = list(filter(lambda x: 'Neighborhood' in x or 'Most' in x, toronto_merged.columns))

In [31]:
def get_cluster_info(df, cluster, select_cols):
    df_cl = df[df['Cluster Labels'] == cluster ][select_cols]
    n_venues = df_cl.shape[0]
    venue_info = '' if n_venues <= 1 else '\n* The 3 Most common venues are:\n\n{}'.format(df_cl.stack().value_counts()[:3])
    print_info = ('Cluster {}:\n\n* Has {} neighborhoods.{}'.format(cluster+1,n_venues,venue_info))
    return df_cl, print_info

### Exploring Clusters:

#### Cluster 1: red

In [32]:
cl1_df, cl1_info = get_cluster_info(toronto_merged, 0, select_cols)
print(cl1_info)

Cluster 1:

* Has 14 neighborhoods.
* The 3 Most common venues are:

Café           13
Coffee Shop    13
Restaurant      8
dtype: int64


In [33]:
cl1_df.style.hide_index()

Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
"Regent Park, Harbourfront",Coffee Shop,Bakery,Park,Pub,Restaurant,Café,Theater
"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop
St. James Town,Café,Coffee Shop,Gastropub,Cocktail Bar,American Restaurant,Italian Restaurant,Gym
Berczy Park,Coffee Shop,Cocktail Bar,Café,Bakery,Italian Restaurant,Farmers Market,Seafood Restaurant
"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Gym,American Restaurant,Clothing Store,Hotel
"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Brewery,Sporting Goods Shop,Fried Chicken Joint
"Toronto Dominion Centre, Design Exchange",Coffee Shop,Hotel,Café,Japanese Restaurant,Restaurant,Salad Place,Deli / Bodega
"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Seafood Restaurant
"University of Toronto, Harbord",Café,Japanese Restaurant,Restaurant,Bar,Italian Restaurant,Bookstore,Bakery
"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Mexican Restaurant,Bakery,Vietnamese Restaurant,Dessert Shop,Bar




#### Cluster 2: purple

In [34]:
cl2_df, cl2_info = get_cluster_info(toronto_merged, 1, select_cols)
print(cl2_info)

Cluster 2:

* Has 1 neighborhoods.


In [35]:
cl2_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
91,Rosedale,Park,Trail,Playground,Creperie,Doner Restaurant,Dog Run,Distribution Center




#### Cluster 3: blue

In [36]:
cl3_df, cl3_info = get_cluster_info(toronto_merged, 2, select_cols)
print(cl3_info)

Cluster 3:

* Has 1 neighborhoods.


In [37]:
cl3_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
25,Christie,Grocery Store,Café,Park,Coffee Shop,Candy Store,Diner,Restaurant



#### Cluster 4: lightgreen

In [38]:
cl4_df, cl4_info = get_cluster_info(toronto_merged, 3, select_cols)
print(cl4_info)

Cluster 4:

* Has 1 neighborhoods.


In [39]:
cl4_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
87,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Coffee Shop,Plane


#### Cluster 5: orange

In [40]:
cl5_df, cl5_info = get_cluster_info(toronto_merged, 4, select_cols)
print(cl5_info)

Cluster 5:

* Has 2 neighborhoods.
* The 3 Most common venues are:

Coffee Shop       2
Sandwich Place    1
Salad Place       1
dtype: int64


In [41]:
cl5_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Diner,Discount Store,Park,Mexican Restaurant,Juice Bar
24,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Thai Restaurant,Bar,Salad Place



### Acknowledgment

This notebook was based on the NY cluseting notebook created by [Alex Aklson](https://www.linkedin.com/in/aklson/) and [Polong Lin](https://www.linkedin.com/in/polonglin/).

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).